In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
mpl.rc("figure", dpi=100)
from matplotlib import pyplot as plt

import numpy as np
from lmz import *


In [ ]:
from lucy import load, adatas

datasets = load.load_scib() + load.load_timeseries()
ssdata = [[adatas.subsample(i,750,31442)  for i in series[:10]]  for series in datasets] 
ssdata = Map(adatas.preprocess, ssdata)




In [ ]:
%%time
'''
for i,da in enumerate(ssdata):
    print("asdasd",i)
    for j,dd in enumerate(da):
        print("asd",j)
        print(sum(map(np.isnan,dd.obs['label'].tolist())) > 0)
        print(sum(map(np.isnan,dd.obs['batch'].tolist())) > 0)
'''

In [ ]:
%%time 
from sklearn.metrics import  silhouette_score
import ubergauss.tools as ug
import time
import lucy.score as lscore
from hyperopt import fmin, tpe, hp, Trials, SparkTrials
from hyperopt.pyll import scope
from ubergauss import hyperopt as uopt


def evalp( x = 0,neighs=10,conns=5, scal=1,comp=5,umaps = 0,outl = .75,
          scalecon = .25,  pca = 30, connect = 100, nalg = 0):
    data = [z.copy() for z in ssdata[x]]
    data = adatas.pca(data,dim = pca, label = 'pca')
    dataset_adjacency = adatas.embed.make_adjacency(adatas.similarity(data), nalg, connect)
    lsa_graph = adatas.lapgraph(data,base = 'pca',
                                              intra_neigh = neighs, 
                                              inter_neigh = conns, 
                                              scaling_num_neighbors = scal,
                                              outlier_threshold = outl,
                                              scaling_threshold = scalecon,
                                              dataset_adjacency =  dataset_adjacency)#(adatas.similarity(data), connect, nalg)) # adjacency_matrix -> symmetrise and binarize 
    data = adatas.graph_embed(data,lsa_graph,n_components = comp, label = 'lsa')
    data = adatas.stack(data)
    
    y = data.obs['label'].tolist()
    ybatch = data.obs['batch'].tolist()
    sim = data.obsm['lsa']
    #return sim, dataset_adjacency
    score = lscore.neighbor_labelagreement(sim,y,5)+2*silhouette_score(sim,y)-lscore.neighbor_labelagreement(sim,ybatch,5)
    return -score


space3 = { 
    #'neighs' : scope.int(hp.quniform('neighs',10,25,1)),
    #'conns' : scope.int(hp.quniform('conns',1,5,1)),
    'scal' : scope.int(hp.quniform('scal',1,5,1)),
    'comp' : scope.int(hp.quniform('comp',4,30,1)),
    # 'umaps' : scope.int(hp.quniform('umaps',0,20,5)),  #  
    'scalecon' : hp.uniform('scalecon',.05,.95),
    'outl' : hp.uniform('outl',.5,.95),
    # 'nalg' : hp.choice ('nalg',[True, False]),
    # 'pca' : scope.int(hp.quniform('pca',30,50,1)),
    'connect' : scope.int(hp.quniform('connect',30,100,1))
}

trials = uopt.fffmin(evalp, 
                     items= Range(ssdata),
                     probing_evals = 30, 
                     probing_parallel = 3,  
                     after_evals = 150, space=space3)

'''
def xx(n):
    try:
        evalp(n)
    except:
        print(n, "FAILED")
ug.xmap(xx,Range(ssdata))
'''

In [ ]:
asd = [t.best_trial['misc']['vals'] for t in trials]

In [ ]:
import pandas as pd 
pd.DataFrame(asd)

In [ ]:
for trial in trials:
    df = uopt.trial2df(trial)
    eeeh = np.argpartition(df['loss'], 10)[:10]
    print(df.iloc[eeeh]) 
    
    

In [ ]:
df[df['loss']< -.04]

In [ ]:
jjj